In [1]:
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine, func
connection_string = "admin22:12345@localhost:5432/saccrewproject2"
engine = create_engine(f'postgresql://{connection_string}')

In [2]:
# Import Walmart 2019 Data
Walmart2019_df = "marketing_sample_for_walmart_com-ecommerce__20191201_20191231__30k_data.csv"
Walmart2019_df = pd.read_csv(Walmart2019_df)
Walmart2019_df = Walmart2019_df.rename(columns = {'List Price':'List Price 2019','Sale Price':'Sale Price 2019'}) #rename columns


In [3]:
#Create dataframe with only desired columns
Walmart2019_clean_df = Walmart2019_df[["Item Number","Product Name","Product Url","List Price 2019","Sale Price 2019","Brand","Category"]]


In [4]:
#Check if any columns have NaN values 
Walmart2019_clean_df.isna().any()

Item Number         True
Product Name       False
Product Url        False
List Price 2019    False
Sale Price 2019    False
Brand               True
Category            True
dtype: bool

In [5]:
#Drop the rows with NaN values
Walmart2019_clean_df = Walmart2019_clean_df.dropna()

In [6]:
# Import Walmart 2020 Data
Walmart2020_df = "marketing_sample_for_walmart_com-product_details__20200101_20200331__30k_data.csv"
Walmart2020_df = pd.read_csv(Walmart2020_df)
Walmart2020_df = Walmart2020_df.rename(columns = {'List Price':'List Price 2020','Sale Price':'Sale Price 2020'}) #rename columns


In [7]:
#Create dataframe with only desired columns
Walmart2020_clean_df = Walmart2020_df[["List Price 2020","Sale Price 2020","Item Number"]]


In [8]:
#Check if any columns have NaN values 
Walmart2020_clean_df.isna().any()

List Price 2020    False
Sale Price 2020    False
Item Number         True
dtype: bool

In [9]:
#Drop the rows with NaN values
Walmart2020_clean_df = Walmart2020_clean_df.dropna()


In [10]:
# Merge Item
Walmart_merged_df = pd.merge(Walmart2019_clean_df, Walmart2020_clean_df, on=['Item Number'], how="inner")


In [11]:
Walmart_merged_df2 = pd.DataFrame(Walmart_merged_df.Category.str.split('|',1).tolist(), columns = ['category1','category2'])




In [12]:
Walmart_merged_df = pd.merge(Walmart_merged_df,Walmart_merged_df2, left_index=True, right_index=True)


In [13]:
Walmart_merged_clean_df = Walmart_merged_df[["Item Number","Product Name","Product Url","List Price 2019","List Price 2020","Brand","category1"]]


In [14]:
Walmart_merged_clean_df = Walmart_merged_clean_df.loc[((Walmart_merged_clean_df["List Price 2019"]>0) & (Walmart_merged_clean_df["List Price 2020"]>0))]

In [15]:
Walmart_merged_clean_df = Walmart_merged_clean_df.rename(columns = {
    'List Price 2019':'price_2019',
    'List Price 2020':'price_2020',
    'Item Number': 'item_number',
    'Product Name': 'product_name',
    'Product Url': 'product_url',
    'Brand': 'brand',
    'category1': 'category'
})

In [16]:
Walmart_merged_clean_df['price_difference']=(Walmart_merged_clean_df['price_2019']-Walmart_merged_clean_df['price_2020'])
Walmart_merged_clean_df['price_percent_change']=(Walmart_merged_clean_df['price_difference']/Walmart_merged_clean_df['price_2019'])*100

In [17]:
Walmart_merged_clean_df = Walmart_merged_clean_df.round(2)

In [18]:
Walmart_merged_clean_df.to_sql(name = "walmartdata", con = engine, if_exists = "append", index = False)

In [19]:
pd.read_sql_query("select * from walmartdata", con = engine).head()

,item_number,product_name,product_url,price_2019,price_2020,price_difference,price_percent_change,brand,category
0,554298081.0,Realtree Ladies Softshell Jacket,https://www.walmart.com/ip/Realtree-Ladies-Sof...,75.22,91.94,-16.72,-22.23,Mossy Oak,Sports & Outdoors
1,563412534.0,"Sechlers Hungarian Sweet-Mild Pepper Relish, 1...",https://www.walmart.com/ip/Sechler-s-Hungarian...,30.11,29.92,0.19,0.63,Sechlers,Food
2,554946697.0,Stealth Cam STCG34 G Series Trail Camera 12 MP...,https://www.walmart.com/ip/Stealth-Cam-STCG34-...,101.59,106.42,-4.83,-4.75,Stealth Cam,Sports & Outdoors
3,555023216.0,"Odor Eaters Stink Stoppers for Kids and Teens,...",https://www.walmart.com/ip/Odor-Eaters-Stink-S...,4.23,4.23,0.00,0.00,Odor-Eaters,Pets
4,560814213.0,Head Jr Alum Tennis Rckt 21In,https://www.walmart.com/ip/Head-Jr-Alum-Tennis...,18.55,18.55,0.00,0.00,Generic,Sports & Outdoors
